# Marangoni boat project code

Welcome to this notebook with code to be used for the data analysis of the boats :) This notebook contains code adapted from original code written by Jackson Wilt and Nico Schramma.
<br />
This notebook contains all data you can use for analysis of the boats, with some comments on how it works. Feel free to copy the notebook and make changes or adaptations however you see fit.

In [ ]:

# Importing relevant packages
import cv2
import numpy as np
import pims
import trackpy as tp
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from copy import copy

## Loading in video
In the cell below you can put the name of the video you want to be analysed. Make sure you put the video in the same folder as where the file with this code is put.

In [2]:
fileName = r'Tracking_videos\7-4-2025\nozzle_0.1mmTOL.mp4'

In [3]:
video = pims.Video(fileName) # Loading in video and converting them to individual frames
duration = video.duration # Duration of video in seconds
frameRate = video.frame_rate # Framerate of video

print(f'duration:{round(duration,3)}, framerate: {round(frameRate,3)}, # of frames: {round(duration * frameRate)}')

duration:38.669, framerate: 29.998, # of frames: 1160


Use the snippet of code underneath if you want to only analyse a selection of frames. When working with large videos it is advisable to first test your code on a small selection of frames. This way you can quickly see if everything works as expected!

In [18]:
startframe = 180
endframe = 900
frames = video[startframe:endframe]
print('Amount of frames:', len(frames))

Amount of frames: 720


In [19]:
def boatCentre(frames):
    """
    Args:
        frames to be analysed

    Returns:
       positions [np.array]: list of frames with only the positions of the tracking dots highlited.
       blurs [np.array]: list of frames with the results of the blurring and clipping shown.
       newset [np.array]: list of original frames with the tracked dots highlited with circles.
       circstore [list]: list with lists of tuples containing the information (pos & radius) of each circle per frame.
       indecesused [list]: list of the indeces corresponding to the frames that were processed from the frames input.
    """
    circstore = [] # list of lists with tuples containing the info of the circles per frame
    shorten = 1 #10
    reducefps = 12
    colorthreshold = 0.35 # threshold value for when to count the pixel as a tracking cap 0 means everything counts
    positions, blurs, newset = [], [], [] #lists to be filled with their respective frame types
    indecesused = []

    for i in range(0,int(len(frames)/shorten),reducefps):
        indecesused.append(i)
        cimg= copy(frames[i]) # makes a copy of the original frame as to not edit over the original

        r,gr,b = cv2.split(frames[i]) # split the color image into different channels    
        blurthis = (abs((r/255)-(gr/255))+abs((gr/255)-(b/255))+abs((r/255)-(b/255))) # calculate the color difference for each pixel from 0-1 (this highlights pixels with a lot of a single color)

        blurthis[blurthis>=colorthreshold] = 255 # set color difference values above a threshold to max intensity
        blurthis[blurthis<colorthreshold] = 0 # set color difference values below the same threshold to zero intenisty
        blur = cv2.medianBlur(np.uint8(blurthis), ksize=7) # blur to help houghcircles. If all went well, only the colored dots on the cheerioboat remain.

        circles = cv2.HoughCircles(blur, cv2.HOUGH_GRADIENT, dp=1.5, minDist=35, param1=60, param2=10, minRadius=2, maxRadius=20) # actually find the circles in the edited image 
        circles = np.uint16(np.around(circles)) # round the found circle positions and radii
        
        circstore.append(circles) # save the positions and radii of the circles
        # blur = cv2.cvtColor(blur,cv2.COLOR_GRAY2RGB)

        background = np.zeros_like(blur, np.uint8) # create a black greyscale background 
    
        for j in circles[0,:]:
            # draw the centers of the circles
            cv2.circle(background,(j[0],j[1]),3,(255,0,0),-1) 
            cv2.circle(blur,(j[0],j[1]),3,255, -1) 
            cv2.circle(cimg,(j[0],j[1]),3,(255,0,0),-1) 
            
            # draw the edge of the circles
            # cv2.circle(background,(i[0],i[1]),i[2],(0,255,0),1)
            cv2.circle(blur,(j[0],j[1]),j[2],(0,255,0),1)
            cv2.circle(cimg,(j[0],j[1]),j[2],(0,255,0),1)
        
        positions.append(background)
        blurs.append(blur)
        newset.append(cimg)

        print('finished frame:', i)
    
    return np.array(positions), np.array(blurs), np.array(newset), circstore, indecesused

## Editing all frames

In [43]:
positions, _, newset, _, indecesused,= boatCentre(frames) # returns frames with dots placed in the centre of mass of the boats for all frames

# print('aantal cirkels:', int(len(np.ndarray.flatten(np.array(circstore)))/3))

finished frame: 0
finished frame: 12
finished frame: 24
finished frame: 36
finished frame: 48
finished frame: 60
finished frame: 72
finished frame: 84
finished frame: 96
finished frame: 108
finished frame: 120
finished frame: 132
finished frame: 144
finished frame: 156
finished frame: 168
finished frame: 180
finished frame: 192
finished frame: 204
finished frame: 216
finished frame: 228
finished frame: 240
finished frame: 252
finished frame: 264
finished frame: 276
finished frame: 288
finished frame: 300
finished frame: 312
finished frame: 324
finished frame: 336
finished frame: 348
finished frame: 360
finished frame: 372
finished frame: 384
finished frame: 396
finished frame: 408
finished frame: 420
finished frame: 432
finished frame: 444
finished frame: 456
finished frame: 468
finished frame: 480
finished frame: 492
finished frame: 504
finished frame: 516
finished frame: 528
finished frame: 540
finished frame: 552
finished frame: 564
finished frame: 576
finished frame: 588
finished f

In [44]:
%matplotlib tk
fig, axs = plt.subplots(1,2, figsize=(10,6))

axs[0].imshow(newset[4])
axs[0].set_title('Frame')
axs[1].imshow(positions[5])
axs[1].set_title('Centre point of boats')

plt.show()

The cell below will search for all the dots we put in the frames and save their positions.

In [45]:
f = tp.batch(positions, diameter=5, minmass=20, processes='auto', invert=False)#37

Frame 59: 3 features


We've now got locations for all our dots! But to get information on the movement of the boats it is important that we can label boats. The code snippet below will compare all our dots frame by frame and detect which ones are which boat.

In [46]:
search_range = 140 
memory       = 20

t = tp.link(f, search_range=search_range, memory=memory)# WAS 90
# t1 = tp.filter_stubs(t, 50)
# t1['t']=t1['frames']

Frame 59: 3 trajectories present.


In [47]:
t

,y,x,mass,size,ecc,signal,raw_mass,ep,frame,particle
0,884.856934,392.856934,398.560505,1.395794,0.217477,49.625454,2805.0,0.0,0,0
1,907.856934,424.856934,398.560505,1.395794,0.217477,49.625454,2805.0,0.0,0,1
2,1411.856934,1426.856934,398.560505,1.395794,0.217477,49.625454,2805.0,0.0,0,2
3,866.856934,335.856934,398.560505,1.395794,0.217477,49.625454,2805.0,0.0,1,0
4,901.856934,349.856934,398.560505,1.395794,0.217477,49.625454,2805.0,0.0,1,1
...,...,...,...,...,...,...,...,...,...,...
174,656.856934,625.856934,398.560505,1.395794,0.217477,49.625454,2805.0,0.0,58,1
175,1411.856934,1426.856934,398.560505,1.395794,0.217477,49.625454,2805.0,0.0,58,2
176,629.856934,599.856934,398.560505,1.395794,0.217477,49.625454,2805.0,0.0,59,0
177,658.856934,626.856934,398.560505,1.395794,0.217477,49.625454,2805.0,0.0,59,1


We've now got the trajectory of our boats! :D

In [48]:
plt.axes().set_aspect('equal')
if t['particle'].max() != 0:
    for i in np.arange(0, t['particle'].max()):
        tt = t.loc[t['particle']==i]
        plt.plot(tt['x'], tt['y'], label = i)
else:
    tt = t.loc[t['particle']==0]
    plt.scatter(tt['x'], tt['y'], c = tt['frame'], s = 0.7)
    plt.plot(tt['x'], tt['y'], color = 'black', zorder = -10, linewidth = 0.5)
    
plt.imshow(frames[0], zorder = -20)
plt.legend()

In [50]:
if t['particle'].max() != 0:
    for i in np.arange(0, t['particle'].max()):
        tt = t.loc[t['particle']==i]
        plt.plot(tt['frame'], np.sqrt(tt['y']**2 + tt['x']**2), label = i)
else:
    plt.plot(tt['frame'], np.sqrt(tt['y']**2 + tt['x']**2), label = 0)
plt.legend()

## Average speed

In [67]:
displacement = []
averageSpeed = []
for i in np.arange(0, t['particle'].max()):
    tt = t.loc[t['particle'] == i]
    tt = tt.reset_index()
    S = 0 
    for j in range(1, len(tt['x'])):
        ds = np.sqrt((tt['x'].iloc[j] - tt['x'].iloc[j - 1])**2 + (tt['y'].iloc[j] - tt['y'].iloc[j - 1])**2)
        S = S + ds
    displacement.append(S)
    averageSpeed.append(S/duration)

In [68]:
data = {
    'particle':  np.arange(t['particle'].max()),
    'displacement' : displacement,
    'averageSpeed' : averageSpeed,
    
}

df = pd.DataFrame(data)

In [69]:
display(df)

,particle,displacement,averageSpeed
0,0,2027.372306,52.429029
1,1,1585.450332,41.000669


In [70]:
import skimage
from skimage import filters, morphology, color, transform, restoration, feature,measure
from skimage.feature import peak_local_max
from scipy import ndimage as ndi
from skimage.segmentation import watershed

In [71]:
i=0
particle_num=1
radius = [15]
thresh=None
plt.figure()
plt.imshow(frames[i])
plt.show()

r,gr,b = frames[i][:,:,0],frames[i][:,:,1],frames[i][:,:,2]
red=(abs((r/np.max(r))-(gr/np.max(gr)))+abs((gr/np.max(gr))-(b/np.max(b)))+abs((r/np.max(r))-(b/np.max(b)))) #map colors to red -> white, else->black
gre=(abs((gr/np.max(gr))-(r/np.max(r)))-abs((r/np.max(r))-(b/np.max(b)))+abs((gr/np.max(gr))-(b/np.max(b)))) #map colors to set green -> white
blu=abs((r/np.max(r))-(b/np.max(b)))+abs((gr/np.max(gr))-(b/np.max(b)))-abs((gr/np.max(gr))-(r/np.max(r))) #map colors to set blue -> white

red = filters.gaussian(red,sigma=3)
if thresh is None:
    thresh = skimage.filters.threshold_otsu(red)
    
binary_red = np.zeros(np.shape(red))
binary_red[red>thresh]=1

area = np.sum(binary_red)
if radius is None:
    radius = [np.sqrt(area/np.pi)] #create radius by the pixle size of the masked object

circles = transform.hough_circle(binary_red,radius=radius)
accums, cx, cy, rad = transform.hough_circle_peaks(circles, radius, total_num_peaks=particle_num)

plt.figure()
plt.imshow(binary_red)
plt.plot(cx,cy,marker='*',c='r')
plt.show()

#check blue and green channels
for i,im in enumerate([gre,blu]):
    im = filters.gaussian(im,sigma=3)
    im_mask = np.zeros(np.shape(im))
    print(cx)
    for cxi,cyi in zip(cx,cy):
        im_mask[np.max([0,int(cyi-radius)]):np.min([int(cyi+radius),np.shape(im)[0]]),np.max([0,int(cxi-radius)]):np.min([int(cxi+radius),np.shape(im)[1]])]=1 #look only around partice positions
    im*=im_mask
    thresh_peak = filters.threshold_otsu(im)
    coord = peak_local_max(im, min_distance=int(radius[0]),threshold_abs=thresh_peak,num_peaks=particle_num)
    print(coord)
    plt.figure()
    plt.imshow(im[0:100,800:1000])
    plt.plot(coord[0,1]-800,coord[0,0],marker='*',color='r')
    plt.show()

[1416]
[]


C:\Users\tiesb\AppData\Local\Temp\ipykernel_1960\368432111.py:39: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  im_mask[np.max([0,int(cyi-radius)]):np.min([int(cyi+radius),np.shape(im)[0]]),np.max([0,int(cxi-radius)]):np.min([int(cxi+radius),np.shape(im)[1]])]=1 #look only around partice positions


IndexError: index 0 is out of bounds for axis 0 with size 0

In [72]:
def spot_detection_3points(frames,shorten=1,reducefps=12,radius=[40],thresh=None,particle_num=1):
    """This script reads frames from a .mp4 file, transforms channels, 
    such that blue and red are mapped to dark, black is mapped to white. 
    Filtering, thresholding, Hough Circle transform
    Final positions are centres of the Hough Circles

    Args:
        file_name (Path) : Path to raw data (.mp4 file)
        shorten   (int, default: 1)  : quotient by which time series is shortened
        reducefps (int, default: 12)  : every Nth step only used, 
        radius (list of int, default: [30,32]) : Radii to test for Hough transform

    Returns:
        pandas Dataframe: with positions 'x','y' and time 'frame','t'
    """
    x=[]
    y=[]
    xg=[]
    xb=[]
    yg=[]
    yb=[]
    radii=[]
    a=[]
    t=[]
    for i in range(0,int(len(frames)/shorten),reducefps):
        r,gr,b = frames[i][:,:,0],frames[i][:,:,1],frames[i][:,:,2]
        red=(abs((r/np.max(r))-(gr/np.max(gr)))+abs((gr/np.max(gr))-(b/np.max(b)))+abs((r/np.max(r))-(b/np.max(b)))) #map colors to red -> white, else->black
        gre=(abs((gr/np.max(gr))-(r/np.max(r)))-abs((r/np.max(r))-(b/np.max(b)))+abs((gr/np.max(gr))-(b/np.max(b)))) #map colors to set green -> white
        blu=-abs((gr/np.max(gr))-(r/np.max(r)))+abs((r/np.max(r))-(b/np.max(b)))+abs((gr/np.max(gr))-(b/np.max(b))) #map colors to set blue -> white
        red = filters.gaussian(red,sigma=2)
        
        if thresh is None:
            thresh = skimage.filters.threshold_otsu(red)
        binary_red = np.zeros(np.shape(red))
        binary_red[red>thresh]=1   
        
        area = np.sum(binary_red)/particle_num
        
        if radius is None:
            radius = [np.sqrt(area/np.pi)] #create radius by the pixle size of the masked object

        circles = transform.hough_circle(binary_red,radius=radius)
        accums, cx, cy, rad = transform.hough_circle_peaks(circles, radius, total_num_peaks=particle_num)

        #check blue and green channels
        for j,im in enumerate([gre,blu]):
            im = filters.gaussian(im,sigma=3)
            im_mask = np.zeros(np.shape(im))
            for cxi,cyi in zip(cx,cy):
                im_mask[np.max([0,int(cyi-radius)]):np.min([int(cyi+radius),np.shape(im)[0]]),np.max([0,int(cxi-radius)]):np.min([int(cxi+radius),np.shape(im)[0]])]=1 #look only around partice positions
            im*=im_mask
            thresh_peak = filters.threshold_otsu(im)
            
            
#             plt.figure()
#             plt.imshow(im)
#             plt.show()
            
            
            coord = peak_local_max(im, min_distance=int(radius[0]),threshold_abs=thresh_peak,num_peaks=particle_num)
            
            
#             print(coord)
#             plt.figure()
#             plt.imshow(im[:200,800:1000])
#             plt.plot(coord[0,1]-800,coord[0,0],marker='*',color='r')
#             plt.show()
            
            
            if j==0:
                xg.extend(coord[:,1])
                yg.extend(coord[:,0])
            else:
                xb.extend(coord[:,1])
                yb.extend(coord[:,0])
        x.extend(cx)
        y.extend(cy)
        radii.extend(rad)
        a.extend(accums)
        t.extend([i/reducefps]*particle_num)
#     print(t)

#     print(len(y))
    p = pd.DataFrame({'y': [*y],
                      'x': [*x],
                      'xb': [*xb],
                      'xg': [*xg],
                      'yb': [*yb],
                      'yg': [*yg],
                      'frame': [*t] ,
                      't': [*t]})
    
    return p

In [73]:
len(frames)

720

In [ ]:
p = spot_detection_3points(frames,  thresh = 0.9,radius = None, reducefps=1)

C:\Users\tiesb\AppData\Local\Temp\ipykernel_1960\3874495728.py:60: RuntimeWarning: When min_distance < 1, peak_local_max acts as finding image > max(threshold_abs, threshold_rel * max(image)).
  coord = peak_local_max(im, min_distance=int(radius[0]),threshold_abs=thresh_peak,num_peaks=particle_num)


In [ ]:
plt.scatter(p['x'], p['y'], s = 5, color = 'black')
plt.scatter(p['xb'], p['yb'],s = 0.9, color = 'blue')
plt.scatter( p['xg'],p['yg'], s = 0.9, color = 'green')
plt.imshow(frames[0])

In [ ]:
fig, ax = plt.subplots()
ax.scatter(p['xg'], p['yg'], color = 'lawngreen')
ax.scatter(p['xb'], p['yb'], color = 'dodgerblue')
ax.scatter(p['x'], p['y'], color = 'red')
ax.set_aspect('equal', adjustable='box')
ax.set_xlabel('x')
ax.set_ylabel('y')

In [ ]:
def displacement(dataframe):
    totalFrames = len(dataframe['t'])
    displacementList = []
    
    for i in np.arange(0, totalFrames - 1):
        data1 = dataframe.loc[dataframe['frame'] == i]
        data2 = dataframe.loc[dataframe['frame'] == i + 1]
        
        ds = np.sqrt( (data2.iloc[0]['x']-data1.iloc[0]['x'])**2  + (data2.iloc[0]['y']-data1.iloc[0]['y'])**2)

        displacementList.append(ds)
        
    return displacementList

In [ ]:
displacementList = displacement(p)

In [ ]:
displacementList

In [ ]:
plt.scatter(p['frame'][:-1],displacementList)
plt.title('distance')
plt.show()

In [ ]:
def angle(dataframe):
    totalFrames = len(dataframe['t'])
    angleList = []
    
    for i in np.arange(0, totalFrames):
        data = dataframe.loc[dataframe['frame'] == i]
        dx = data.iloc[0]['xb'] - data.iloc[0]['xg']
        dy = data.iloc[0]['yb'] - data.iloc[0]['yg']
        angle = np.arctan(dy/dx)
        angleList.append(angle)
        
    return angleList

In [ ]:
angles = angle(p)

In [ ]:
plt.plot(p['frame'], angles)
plt.xlabel('frame')
plt.ylabel('angle')
plt.show()

In [ ]:
plt.plot(p['frame'],p['yg'], color = 'lawngreen')
plt.plot(p['frame'],p['yb'], color = 'dodgerblue')
plt.plot()